In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,SimpleRNN
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:

# Load dataset
dataset = pd.read_csv("../../dataset/Dataset_Iov.csv")

# Extract features and target
features = ['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']
target = 'Collussion'

X = dataset[features].values
y = dataset[target].values

# Normalize features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)


In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# # Define sequence length and batch size
# num_time_steps = 1  # Steps per training iteration
# batch_size = 128  # Batch size for training
# num_features = X.shape[1]

# def create_sequences(X, y, time_steps):
#     sequences, labels = [], []
#     for i in range(len(X) - time_steps):
#         sequences.append(X[i:i + time_steps])
#         labels.append(y[i + time_steps])  # Predict next step
#     return np.array(sequences), np.array(labels).astype(np.float32).reshape(-1,)  # Ensure binary labels are float32


# Create sequences
# X_seq, y_seq = create_sequences(X, y, num_time_steps)
X=X.reshape(X.shape[0],1,X.shape[1])

# Split dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=42, shuffle=False)

# Check dataset shape
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")  
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (186097, 1, 6), y_train shape: (186097,)
X_test shape: (20678, 1, 6), y_test shape: (20678,)


In [81]:
print(X_train[:3])
print(X_train.shape, y_train.shape)

[[[ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
   -0.35360533]
  [ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]
  [ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]]

 [[ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]
  [ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]
  [ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]]

 [[ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]
  [ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]
  [ 0.54556369 -1.48075905 -0.4296399  -0.49175573 -0.76073707
    2.82801171]]]
(186094, 3, 6) (186094,)


In [82]:
# Model Configuration
num_hidden_layers = 3
num_neurons = 100


In [83]:

# Building the Model
model = Sequential()

# First LSTM layer
from tensorflow.keras.layers import Input

model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(SimpleRNN(num_neurons, return_sequences=True))  

# Additional LSTM layers
for _ in range(num_hidden_layers - 2):
    model.add(LSTM(num_neurons, return_sequences=True))

# Last LSTM layer without return_sequences
model.add(LSTM(num_neurons))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model (Adam optimizer, MSE loss)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mse'])

In [84]:
# Model Summary
print(model.summary())

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_6 (SimpleRNN)        │ (None, 3, 100)         │        10,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 3, 100)         │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 171,601 (670.32 KB)

 Trainable params: 171,601 (670.32 KB)

 Non-trainable params: 0 (0.00 B)

None


In [85]:
# Train the model with 50 epochs and batch size 32
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/50
1454/1454 ━━━━━━━━━━━━━━━━━━━━ 46s 25ms/step - loss: 0.6347 - mse: 0.2215 - val_loss: 0.6345 - val_mse: 0.2215
Epoch 2/50
1176/1454 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - loss: 0.6277 - mse: 0.2183

KeyboardInterrupt: 

In [ ]:
# Predictions
y_pred = model.predict(X_test)

# Convert categorical predictions to labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Calculate MSE and RMSE
mse = mean_squared_error(y_test_labels, y_pred_labels)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")